# Training ```FFANN```

Let's augment the ```FFANN``` class in order to be able to train it.

This is still a work in progress, but at least it can "fit" the *XOR*-gate and a linear function.

In [1]:
import FeedForwardANN as FFANN
import numpy as np

In [2]:
lin=FFANN.linearActivation()
sig=FFANN.sigmoidActivation()

In [3]:
# data_in=[[_] for _ in np.linspace(0,1,250)]
# data_out=[[2*_] for _ in np.linspace(0,1,250)]
# brain=FFANN.FFANN(1,1,[3,2,1],[lin,lin,lin,lin,lin])
# brain.init_params(-1,1)


data_in=[[1,1],[0,0],[1,0],[0,1]]
data_out=[[0],[0],[1],[1]]
brain=FFANN.FFANN(2,1,[3],[sig,lin])
brain.init_params(-1e-1,1e-1)

In [4]:
#the squared error and its derivative
def Q_i(signal, target):
    return (signal-target)**2


def dQds_i(signal, target):
    return 2*(signal-target)

In [5]:
Q=FFANN.loss(data_in, data_out,Q_i, dQds_i, brain)
# SGD=FFANN.VanillaSGD(Q,alpha=1e-1)
# SGD=FFANN.RMSpropSGD(Q,gamma=0.995,epsilon=1e-4,alpha=1e-2)
# SGD=FFANN.AdaDeltaSGD(Q,gamma=0.995,epsilon=1e-4,alpha=1)
# SGD=FFANN.AdamSGD(Q,beta_m=1-1e-1,beta_v=1-1e-3,epsilon=1e-8,alpha=1e-2)
SGD=FFANN.AdaMaxSGD(Q,beta_m=1-1e-1,beta_v=1-1e-3,epsilon=1e-8,alpha=1e-2)
# SGD=FFANN.NAdamSGD(Q,beta_m=1-1e-1,beta_v=1-1e-3,epsilon=1e-8,alpha=1e-2)

In [ ]:
SGD.run(abs_tol=1e-3, rel_tol=1e-3, step_break=250,max_step=50000)

In [ ]:
meanQ=0
maxQ=0
for i,_ in enumerate(data_in):
    brain(_)
    Qi=Q(data_out[i])
    print('point:',_,'\n'
       'target:',data_out[i],
         'FFANN gives:',brain(_),
         'loss:',Qi)
    print('')
    if Qi > maxQ:
        maxQ=Qi
    brain(_)
    meanQ+=Q(data_out[i])
    
print('mean loss:', meanQ/float(len(data_out)))
print('max loss:', maxQ)